In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Step 1: Read the Excel file
file_path = r"C:\Users\CHAKU FOODS\Documents\Data Collection Cleaning\Irrigation.xlsx" 
data = pd.read_excel(file_path)

# Ensure the column names in the Excel file match the database table structure
# For example, check if columns are ['last_name', 'first_name', 'relies_on_rainfall', 'other_irrigation_methods']

# Step 2: Establish connection to the MySQL database
try:
    connection = mysql.connector.connect(
        host='localhost',  # Change to your host
        user='root',  # Replace with your MySQL username
        password='KpakpoParker700!',  # Replace with your MySQL password
        database='chaku_foods'  # Replace with your database name
    )

    if connection.is_connected():
        cursor = connection.cursor(dictionary=True)

        # Step 3: Retrieve farmer IDs based on last_name and first_name
        for index, row in data.iterrows():
            last_name = row['last_name']
            first_name = row['first_name']

            # Query to get farmer_id
            cursor.execute(
                """
                SELECT farmer_id FROM farmer WHERE last_name = %s AND first_name = %s
                """, (last_name, first_name)
            )
            result = cursor.fetchone()

            if result:
                farmer_id = result['farmer_id']

                # Step 4: Prepare the insert statement
                insert_query = """
                INSERT INTO irrigation (
                    farmer_id, relies_on_rainfall, other_irrigation_methods
                ) VALUES (%s, %s, %s)
                """

                cursor.execute(
                    insert_query,
                    (
                        farmer_id,
                        row['relies_on_rainfall'] if pd.notna(row['relies_on_rainfall']) else None,
                        row['other_irrigation_methods'] if pd.notna(row['other_irrigation_methods']) else None
                    )
                )

        # Step 5: Commit the transaction
        connection.commit()
        print("Data imported successfully!")

except Error as e:
    print(f"Error: {e}")

finally:
    # Step 6: Close the connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")


Data imported successfully!
MySQL connection closed.
